In [1]:

import numpy as np
import matplotlib.pyplot as plt
import math
import ipympl
from timeit import default_timer as timer

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [3]:
from load_data import load_file
from helper_functions import calc_scaling,symmetrize,rotate_to_reference,single_rotD,single_rotP
from calc_average import full_average,full_average_IR,full_average_R,full_average_R_orth,numerical_sector_average
from calc_single import oriented_IR,oriented_R,single_rot_IR,single_rot_R

In [4]:
from plotting_functions import create_average_spec_single

In [5]:
#filename="freq-19813-90-2.out"   # less accurate
filename="freq-19813-90-2.fchk"   # more accurate
fr,Z,Q,D,P0,nat,aniso=load_file(filename)

Loaded  freq-19813-90-2.fchk


In [6]:
phys_params=dict(laser =785, #633, 
                 T = 298.15
                 )

# calculate intensity scaling factors
v0= math.pow(10, 7)/phys_params['laser']

scalingIR,scaling,scalingexp= calc_scaling(phys_params['T'])
# print("IR scaling ",scalingIR)
# print("Raman internal scaling ",scaling)
pi=math.pi
torad=2*pi/360

In [24]:
def plot_spectrum_oriented(phi,theta,show_av):
    # modes: list of normal modes to calculate
    modes=range(0,len(fr)) #[31,40]

    rotate=0    # rotate molecule to reference orientation before analysis, NOT IMPLEMENTED YET
    numerical=0 # perform numerical tests
    single=1    # get intensities for single rotations
    

    # rotational angles for single rotations 
    # nx=3      
    # ths=np.linspace(0,pi/2,  num=nx) 
    # phs=np.linspace(0,2*pi,  num=nx)     

    th=theta*torad #[w_theta.value*torad] #np.linspace(0,pi/2,  num=3) 
    ph=phi*torad #[w_phi.value*torad] #np.linspace(0,pi/2,  num=3)

    Lm=1

    # Full orientation averages
    nummodes=len(modes)
    conv_av=np.zeros((nummodes))     # conversion intensity
    ir_av=np.zeros((nummodes))    # IR intensity
    r_av=np.zeros((nummodes))     # Raman Stokes intensity, parallel fields
    r_a_av=np.zeros((nummodes))   # Raman anti-Stokes intensity, parallel fields
    r_av_ort=np.zeros((nummodes)) # Raman Stokes intensity, orthogonal fields

    # Single orientations
    ir_single=np.zeros((nummodes))  # IR intensity
    r_single=np.zeros((nummodes))   # Raman Stokes, parallel fields
    conv_single=np.zeros((nummodes)) # Conversion intensity, parallel fields

    for n,m in enumerate(modes):
     #   print("Mode ",m)
        P=symmetrize(P0[m,:,:])
        if rotate:
            Drot,Prot,R=rotate_to_reference(D[m,:],P,Z,Q)

    # Calculate frequency-dependent scaling factors
        # Usual Stokes for thermal population
        scalingR=Lm*scaling* math.pow(v0 - fr[m], 4) / (
                    fr[m] * (1 - math.exp(scalingexp * fr[m]))) 
        # Usual anti-Stokes for thermal population
        scalingaR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] *(
               1/(-1+math.exp(-scalingexp * fr[m]))) # 
        # For THOR: anti-Stokes without population
        scalingTHOR=Lm*scaling* math.pow(v0 + fr[m], 4) / fr[m] 

    # Calculate full averages analytically
        starta=timer()
        conv_av[n] = scalingIR *scalingTHOR*full_average(D[m,:],P)  
        ir_av[n]=scalingIR*full_average_IR(D[m,:]) 
        Rav=full_average_R(P)
        r_av[n]=scalingR*Rav 
        enda=timer()
        r_a_av[n]=scalingaR*Rav 
        r_ort,depol=full_average_R_orth(P)
        r_av_ort[n]=scalingR*r_ort 

    # Test: Calculate full averages numerically 
        if numerical:
                startm=timer()
                ir_numav, r_numav,p_numav=numerical_sector_average(D[m,:],P,1,1,0,0,0)
                ir_numav=scalingIR*ir_numav
                r_numav=scalingR*r_numav
                p_numav=scalingIR *scalingTHOR*p_numav
                endm=timer()
                print("Time for evaluation\n   Analytic \t{:.1e}s \n   Numerical \t{:.1e}s \n   Num/An \t{:.1e}".format(
                    enda-starta,endm-startm,(endm-startm)/(enda-starta)))
                print("Abs. and rel. error of numerical results\n   IR \t\t{:.2e}\t{:.2f}% \n   Raman \t{:.2e}\t{:.2f}% \n   Conv \t{:.2e}\t{:.2f}%".format(
                    ir_numav-ir_av[n],(ir_numav-ir_av[n])/ir_av[n]*100,r_numav-r_av[n],(r_numav-r_av[n])/r_av[n]*100,
                    p_numav-conv_av[n],(p_numav-conv_av[n])/conv_av[n]*100))
                print("-----------------------------------------------")

    # Calculate single orientations analytically
        if single:
                if numerical:
                    print("IR intensity                            | Raman intensity")
                    print("single    numeric1  numeric2   orient   | single    numeric1   numeric2   orient")
                ir=single_rot_IR(D[m,:],a=ph,b=th,c=0) 
                r=single_rot_R(P,a=ph,b=th,c=0)
                ir_single[n]=scalingIR*ir
                r_single[n]=scalingR*r
                conv_single[n]=scalingIR*scalingTHOR*ir*r

    # Tests: numerical average on rotated D and P should give the same value 
    #             when using nump=1, 
    #             or in the limit of k->inf and l->inf
    #        rotating D and P first and using field oriented along z should also give the same
                if numerical:
                            Drot=single_rotD(D[m,:],a=ph,b=th,c=0)
                            Prot=single_rotP(P,a=ph,b=th,c=0)
                            ir_snumav, r_snumav,p_snumav=numerical_sector_average(Drot,Prot,1,1,0,0,0,nump=1)
                            ir_snumav2, r_snumav2,p_snumav2=numerical_sector_average(Drot,Prot,1000000,1000000,0,0,0,nump=30)
                            print("{:.3e} {:.3e} {:.3e} {:.3e} | {:.3e} {:.3e} {:.3e} {:.3e}".format(ir_single[n],
                                scalingIR*ir_snumav,scalingIR*ir_snumav2,scalingIR*oriented_IR(Drot),
                                r_single[n],
                                scalingR*r_snumav,scalingR*r_snumav2,scalingR*oriented_R(Prot)))
    %matplotlib widget
#    from plotting_functions import plot_spectra

    xmin=30
    xmax=1000
    res=0.5
    gammaIR=5
    gammaR=5
    sclf=0.98                        
    #print("Single orientations")
    title=r'{} $\phi$={:.1f}$\degree$ $\theta$={:.1f}$\degree$'.format(filename.split(".")[0],ph/torad,th/torad)
    wn,R_spec,IR_spec,conv_spec,freqs,prod_ints,R_ints,IR_ints,P,A,R=create_average_spec_single(fr,  ir_single, r_single, 
                                                                                      conv_single,
                                                                                      xmin,xmax,res,gammaIR,gammaR,sclf)
#    plot_spectra(0,wn,R_spec,IR_spec,freqs,prod_ints,xmin,xmax,res,title,0)
    wn,R_spec_av,IR_spec_av,conv_spec_av,freqs,prod_ints_av,R_ints_av,IR_ints_av,P_av,A_av,R_av=create_average_spec_single(fr,  ir_av, r_av, conv_av,
                                                                                  xmin,xmax,res,gammaIR,gammaR,sclf)
#    plot_spectra(0,wn,R_spec,IR_spec,freqs,prod_ints,xmin,xmax,res,filename.split(".")[0],0)
    
    
    plt.rcParams.update({'font.size': 16})
    fig=plt.figure()
    fig.set_size_inches(8, 7)
    ax2=fig.add_subplot(311)
    ax3=fig.add_subplot(312,sharex=ax2)
    ax1=fig.add_subplot(313,sharex=ax2)
    ax3.fill_between(wn,R_spec,alpha=0.6,color='b',label='Raman')
    ax2.fill_between(wn,IR_spec,alpha=0.6,color='r',label='IR') 
    ax1.fill_between(wn,conv_spec,alpha=0.6,color='k',label='conv') 
 #   markerline, stemline, baseline, =ax1.stem(freqs,prod_ints,
 #                                                'k',markerfmt='ok',basefmt='k',
 #                                                use_line_collection=True,bottom=0,label='Conv.')
    pmin=int(xmin/res)
    pmax=int(xmax/res)
    maxpr=np.max(conv_spec[pmin:pmax]) #np.max(prod_ints)
    maxI=np.max(IR_spec[pmin:pmax])
    maxR=np.max(R_spec[pmin:pmax])
    if show_av:
        ax3.fill_between(wn,R_spec_av,alpha=0.6,color='grey',label='Raman av')
        ax2.fill_between(wn,IR_spec_av,alpha=0.6,color='grey',label='IR av')
        ax1.fill_between(wn,conv_spec_av,alpha=0.6,color='grey',label='conv av')
 #       markerline_av, stemline_av, baseline_av, =ax1.stem(freqs,prod_ints_av,
 #                                                'grey',markerfmt='o',basefmt='grey',
 #                                                use_line_collection=True,bottom=0,label='Conv.')
        maxpr=max(maxpr,np.max(conv_spec_av[pmin:pmax])) #np.max(prod_ints_av)
        maxI=max(maxI,np.max(IR_spec_av[pmin:pmax]))
        maxR=max(maxR,np.max(R_spec_av[pmin:pmax]))
 #   plt.setp(stemline, linewidth = 1.25)
 #   plt.setp(markerline, markersize = 6)    
    plt.xlim(xmin,xmax)
    
    ax1.set_ylim(-maxpr/100,1.2*maxpr)
    ax2.set_ylim(-maxI/100,1.2*maxI)
    ax3.set_ylim(-maxR/100,1.2*maxR)
    plt.setp(ax2.get_xticklabels(), visible=False)
    plt.setp(ax3.get_xticklabels(), visible=False)
    ax1.ticklabel_format(axis="y", style="sci", scilimits=(0,0),useMathText=True)
    ax2.ticklabel_format(axis="y", style="sci", scilimits=(0,0),useMathText=True)
    ax3.ticklabel_format(axis="y", style="sci", scilimits=(0,0),useMathText=True)
    ax2.set_title(title)
    plt.xlabel('Wavenumber (cm$^{-1}$)')
    ax2.set_ylabel('IR',color='r')
    ax3.set_ylabel('R',color='b')
    ax1.set_ylabel('Conv.')
    ax1.yaxis.set_label_coords(-0.07,0.5)
    ax2.yaxis.set_label_coords(-0.07,0.5)
    ax3.yaxis.set_label_coords(-0.07,0.5)
    fig.tight_layout(pad=1.8, w_pad=0.01, h_pad=0.01)
    plt.show()
    
    

In [25]:
interact(plot_spectrum_oriented, theta=widgets.BoundedFloatText(value=0,min=0,max=90.0,step=0.1,description='Tilt (x):',disabled=False),
         phi=widgets.BoundedFloatText(value=0,min=0,max=360.0,step=0.1,description='Rotation (z):',disabled=False),
         show_av=False);

interactive(children=(BoundedFloatText(value=0.0, description='Rotation (z):', max=360.0, step=0.1), BoundedFl…

In [23]:
# Test: check if average of single rotations gives full average
# weighting function: pi*sin(theta)/(2*number of single rotations)
# set nx larger for high accuracy
test=0      # test against average of single rotations, needs high nx, theta [0,pi/2], phi [0,2*pi]
if test:
    print("\tfull average  \tav. of single rotations \terror%")
    ir_single_av=np.zeros((nummodes))
    r_single_av=np.zeros((nummodes))
    conv_single_av=np.zeros((nummodes))

    for n,m in enumerate(modes):
        print ("Mode ",m)
        ir_single_av[n]=pi*np.sum(np.sum(ir_single[n,:,:],axis=1)*np.sin(np.array(ths)))/(len(ths)*len(phs)*2)
        print("IR  \t{:.3e} \t\t{:.3e} \t\t{:.3f}%".format(ir_av[n],ir_single_av[n],
                                                          (ir_single_av[n]-ir_av[n])/ir_av[n]))

        r_single_av[n]=pi*np.sum(np.sum(r_single[n,:,:],axis=1)*np.sin(np.array(ths)))/(len(ths)*len(phs)*2)
        print("Raman \t{:.3e} \t\t{:.3e} \t\t{:.3f}%".format(r_av[n],r_single_av[n],
                                                            (r_single_av[n]-r_av[n])/r_av[n]))


        conv_single_av[n]=pi*np.sum(np.sum(conv_single[n,:,:],axis=1)*np.sin(np.array(ths)))/(len(ths)*len(phs)*2)
        print("Conv.  \t{:.3e} \t\t{:.3e} \t\t{:.3f}%".format(
            conv_av[n],conv_single_av[n],(conv_single_av[n]-conv_av[n])/conv_av[n]))